# Sample Propotion

## Variables

* count1: # of success for experiment
* nobs1: size for experiment
* count2: # of success for control
* nobs2: size for experiment
* prop1: the propotion of success for experiment
* prop2: the propotion of success for control
* ratio: nobs2/nobs1
* alpha
* power
* dmin

In [ ]:
#summary: https://sonalake.com/latest/hypothesis-testing-of-proportion-based-samples/

In [6]:
#statsmodel use p1- p2
import statsmodels.stats.power as smspower
import statsmodels.stats.proportion as prop
from scipy.stats import norm
import numpy as np

## 1.Sample Size

In [ ]:
#Calculate sample size using self_defined function

In [78]:
#parameter needed
prop2 = 0.13
dmin = 0.02
power = 0.8
alpha = 0.05

In [79]:
def min_sample_size(prop2, dmin, power=0.8, alpha=0.05):
    """Returns the minimum sample size to set up a split test
    Arguments:
        prop2 (float): probability of success for control, sometimes
        referred to as baseline conversion rate
        dmin (float): minimum change in measurement between control
        group and test group if alternative hypothesis is true, sometimes
        referred to as minimum detectable effect
        power (float): probability of rejecting the null hypothesis when the
        null hypothesis is false, typically 0.8
        sig_level (float): significance level often denoted as alpha,
        typically 0.05
    Returns:
        min_N: minimum sample size (float)
    References:
        Stanford lecture on sample sizes
        http://statweb.stanford.edu/~susan/courses/s141/hopower.pdf
    """

    # find Z_beta from desired power
    Z_beta = norm.ppf(power)

    # find Z_alpha
    Z_alpha = norm.ppf(1-alpha/2)

    # average of probabilities from both groups
    pooled_prob = (prop2 + prop2 + dmin) / 2

    min_N = (2 * pooled_prob * (1 - pooled_prob) * (Z_beta + Z_alpha)**2
             / dmin**2)

    return min_N

In [80]:
min_sample_size(prop2, dmin)

4725.025600078151

In [ ]:
#Calculate sample size using statsmodels

In [81]:
#parameter needed
prop2 = 0.13
dmin = 0.02
prop1 = prop2 + dmin
power = 0.8
alpha = 0.05

In [82]:
#effect
std_effect = prop.proportion_effectsize(prop1, prop2)
#sample size
smspower.zt_ind_solve_power(effect_size = std_effect, nobs1=None, alpha=.05, power=.80)

4719.4740575998185

## 2.Confidence Interval

In [7]:
pageviews_cont=[ 7723,  9102, 10511,  9871, 10014,  9670,  9008,  7434,  8459,
       10667, 10660,  9947,  8324,  9434,  8687,  8896,  9535,  9363,
        9327,  9345,  8890,  8460,  8836,  9437,  9420,  9570,  9921,
        9424,  9010,  9656, 10419,  9880, 10134,  9717,  9192,  8630,
        8970]
pageviews_exp=[ 7716,  9288, 10480,  9867,  9793,  9500,  9088,  7664,  8434,
       10496, 10551,  9737,  8176,  9402,  8669,  8881,  9655,  9396,
        9262,  9308,  8715,  8448,  8836,  9359,  9427,  9633,  9842,
        9272,  8969,  9697, 10445,  9931, 10042,  9721,  9304,  8668,
        8988]
clicks_cont=[687, 779, 909, 836, 837, 823, 748, 632, 691, 861, 867, 838, 665,
       673, 691, 708, 759, 736, 739, 734, 706, 681, 693, 788, 781, 805,
       830, 781, 756, 825, 874, 830, 801, 814, 735, 743, 722]
clicks_exp=[686, 785, 884, 827, 832, 788, 780, 652, 697, 860, 864, 801, 642,
       697, 669, 693, 771, 736, 727, 728, 722, 695, 724, 789, 743, 808,
       831, 767, 760, 850, 851, 831, 802, 829, 770, 724, 710]
enrolls_cont=[134, 147, 167, 156, 163, 138, 146, 110, 131, 165, 196, 162, 127,
       220, 176, 161, 233, 154, 196, 167, 174, 156, 206]
enrolls_exp=[105, 116, 145, 138, 140, 129, 127,  94, 120, 153, 143, 128, 122,
       194, 127, 153, 213, 162, 201, 207, 182, 142, 182]

In [ ]:
#using self_defined function(unpooled SE)

In [83]:
#parameter needed
#experiment:
count1 = sum(clicks_exp)
nobs1 = sum(pageviews_exp)
#control
count2 = sum(clicks_cont)
nobs2 = sum(pageviews_cont)

In [86]:
def two_proprotions_confint(count1, nobs1, count2, nobs2, alpha = 0.05):
    """
    A/B test for two proportions;
    given a success a trial size of group A and B compute
    its confidence interval;

    Parameters
    ----------
    count1, count2 : int
        Number of successes in each group

    nobs1, nobs2 : int
        Size, or number of observations in each group

    alpha : float, default 0.05

    Returns
    -------
    prop_diff : float
        Difference between the two proportion

    confint : 1d ndarray
        Confidence interval of the two proportion test
    """
    prop1 = count1 / nobs1
    prop2 = count2 / nobs2
    var = prop1 * (1 - prop1) / nobs1 + prop2 * (1 - prop2) / nobs2
    se = np.sqrt(var)

    # z critical value
    z = norm.ppf(1-alpha/2)

    # standard formula for the confidence interval
    # point-estimtate +- z * standard-error
    diff = prop1 - prop2
    confint = diff + np.array([-1, 1]) * z * se
    return diff, confint

In [87]:
two_proprotions_confint(count1, nobs1, count2, nobs2, alpha = 0.05)

(5.662709158693602e-05, array([-0.00123903,  0.00135228]))

In [88]:
#use package(pooled SE)
prop.confint_proportions_2indep(count1, nobs1, count2, nobs2,  
                                method=None, compare='diff', alpha=0.05, correction=False)

(-0.0012363851721392553, 0.001349672826183403)

## 3.P Value

In [ ]:
#using package

In [8]:
n=len(enrolls_exp)
#experiment
count1 = sum(enrolls_exp[:n])
nobs1 = sum(clicks_exp[:n])
#control
count2 = sum(enrolls_cont[:n])
nobs2 = sum(clicks_cont[:n])

In [12]:
nobs1/count1

5.042360502483202

In [9]:
#https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.test_proportions_2indep.html#statsmodels.stats.proportion.test_proportions_2indep
prop.test_proportions_2indep(count1, nobs1, count2, nobs2,  
                             value=None, method=None, 
                             compare='diff', alternative='two-sided', 
                             correction=True, return_results=True)

<class 'statsmodels.stats.base.HolderTuple'>
statistic = -4.702930425242798
pvalue = 2.5645382823495675e-06
compare = diff
method = agresti-caffo
diff = -0.020554874580361565
ratio = 0.9060883894007418
odds_ratio = 0.8828565152169403
variance = 1.909806144605413e-05
alternative = two-sided
value = 0
tuple = (-4.702930425242798, 2.5645382823495675e-06)

In [ ]:
#By Hand
#https://medium.com/analytics-vidhya/testing-a-difference-in-population-proportions-in-python-89d57a06254

## 4.Power

In [92]:
#parameter needed
prop1 = count1/nobs1
prop2 = count2/nobs2
diff = prop1 - prop2
ratio = nobs2/nobs1

In [93]:
prop.power_proportions_2indep(diff, 
                              prop2, nobs1, 
                              ratio, alpha=0.05, value=0, 
                              alternative='two-sided', return_results=True)

<class 'statsmodels.tools.testing.Holder'>
power = 0.996933464150996
p_pooled = 0.2085972518904125
std_null = 0.5746032342306825
std_alt = 0.5744193810743623
nobs1 = 17260
nobs2 = 17293.0
nobs_ratio = 1.0019119351100811
alpha = 0.05

# Sample Mean

In [97]:
#https://www.statsmodels.org/stable/generated/statsmodels.stats.power.TTestIndPower.html#statsmodels.stats.power.TTestIndPower

In [98]:
#https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html#statsmodels.stats.power.tt_ind_solve_power